In [4]:
from gpt3forchem.data import get_photoswitch_data
from gpt3forchem.input import create_single_property_forward_prompts

from sklearn.model_selection import train_test_split

In [3]:
data = get_photoswitch_data()

In [6]:
train_data, test_data = train_test_split(data, train_size=0.8, random_state=None, stratify=data["wavelength_cat"])

In [7]:
train_prompts = create_single_property_forward_prompts(
    train_data,
    "wavelength_cat",
    {"wavelength_cat": "transition wavelength"},
    representation_col="SMILES",
    smiles_augmentation=True,
)


In [ ]:
test_prompts = create_single_property_forward_prompts(
    test_data,
    "wavelength_cat",
    {"wavelength_cat": "transition wavelength"},
    representation_col="SMILES",
    smiles_augmentation=True,
)